<a href="https://colab.research.google.com/github/rizkyray03/RVC-AI-VOICE/blob/main/Ultimate_AI_Voice_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Online or Local

OPTION = "HuggingFace" #@param ["HuggingFace", "GoogleDrive"]


In [ ]:
#@title 1.Mount Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2.Install Dependencies
!pip install yt_dlp
!pip install ffmpeg
!mkdir youtubeaudio
%cd /content
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

#@title 3.Clone RVC Github
!git clone --depth=1 -b stable https://github.com/fumiama/Retrieval-based-Voice-Conversion-WebUI

#@title 5.Install aria2
!apt -y install -qq aria2

#@title 6.Download Pretrained Model for Audio Splitting
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

#@title 7.Download hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
#@title 3.Download Youtube Audio
local = '/content/youtubeaudio/'
name = 'cars-outside' #@param {type:"string"}
mix = local + name

from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys



ydl_opts = {
    'format': 'bestaudio/best',
#    'outtmpl': 'output.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }],
    "outtmpl": mix,  # this is where you can edit how you'd like the filenames to be formatted
}
def download_from_url(url):
    ydl.download([url])
    # stream = ffmpeg.input('output.m4a')
    # stream = ffmpeg.output(stream, 'output.wav')


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      url = "https://www.youtube.com/watch?v=xP5MCCnnSS4" #@param {type:"string"}
      download_from_url(url)





In [ ]:
#@title 4.Choose Model
REPOSITORY  = "Harukichi"
def get_model(MODELNAME):
    names = ["cust1",
             "ray",
             "haru"]
    probe = ["added_IVF345_Flat_nprobe_1.index",
             "added_IVF771_Flat_nprobe_1.index",
             "added_IVF90_Flat_nprobe_1.index"]

    if MODELNAME in names:
        index = names.index(MODELNAME)
        return probe[index]
    else:
        return None

MODELNAME = "ray" #@param ["cust1", "ray" , "haru"]
INDEX = get_model(MODELNAME)

print(MODELNAME)
print(INDEX)
URL = f"https://huggingface.co/{REPOSITORY}/{MODELNAME}/resolve/main/weights/{MODELNAME}.pth"
INDEXURL = f"https://huggingface.co/{REPOSITORY}/{MODELNAME}/resolve/main/{MODELNAME}/{INDEX}"

In [ ]:
#@title 5.Download model

if OPTION == 'HuggingFace' :
    !wget -N {URL} -P /content/Retrieval-based-Voice-Conversion-WebUI/weights
    # !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
    !wget -N {INDEXURL} -P /content/{MODELNAME}
    %cd /content/Retrieval-based-Voice-Conversion-WebUI
    !mkdir -p pretrained uvr5_weights
else :
  #@title 4.2 Copy Model to RVC Directory
  !cp -r /content/drive/MyDrive/RVC/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/weights
  !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
  !cp -r /content/drive/MyDrive/RVC/{MODELNAME}/* /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
  %cd /content/Retrieval-based-Voice-Conversion-WebUI
  !mkdir -p pretrained uvr5_weights






In [ ]:
#@title 6.Run Web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3